In [15]:
# Install Java, Spark, and Findspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!apt-get update
!apt-get install openjdk-8-jdk
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1,338 kB]
Fetched 1,590 kB

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLEnvironment").getOrCreate()

In [17]:
from pyspark import SparkFiles
#Import matched_npa_requests.csv file
url_5 = "https://charlotte-rezoning-model.s3.us-east-2.amazonaws.com/matched_npa_requests_year.csv"
spark.sparkContext.addFile(url_5)
npa_data_df = spark.read.csv(SparkFiles.get("matched_npa_requests_year.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
npa_data_df.show()

+---+----------+------------------+---+--------------------+--------+----------+--------------------+----------+----------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------------+----------------+----------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----+
|_c0|Unnamed: 0|         npa_acres|NPA|        npa_geometry|OBJECTID|  Petition|          Petitioner|  FromZone|    ToZone|Type|SPA|Overlay|INNOV|              Acres|           PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ|     CCW|      ServiceArea|       LuseStaff|     UrbanDesign|       RezoneStaff|Consistent|ProtestPet|            Received|            Approved|Decision|    

In [18]:
from pyspark.sql.functions import isnan, isnull, when, count, col

In [19]:
# there are null Decision values....they need to be removed
npa_data_df2 = npa_data_df.withColumn("nullDecisionFlag", when(isnull("Decision"), 0).otherwise(1))
npa_data_df2.filter("nullDecisionFlag = 0").show()
print(npa_data_df2.count())

+---+----------+-------------+---+--------------------+--------+-----------+----------+--------+----------+----+---+-------+-----+-----+--------+-----+-----+---------+---------+----+----+-----------+---------+-----------+-----------+----------+----------+--------+--------------------+--------+---------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+--------------------+------------------+--------------------+-----------+----+----------------+
|_c0|Unnamed: 0|    npa_acres|NPA|        npa_geometry|OBJECTID|   Petition|Petitioner|FromZone|    ToZone|Type|SPA|Overlay|INNOV|Acres|PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ| CCW|ServiceArea|LuseStaff|UrbanDesign|RezoneStaff|Consistent|ProtestPet|Received|            Approved|Decision|Hyperlink|Corrective|Propluse|AdminAppr|AdminApprD|AdminApprP|AdminApprC|SFLOTS|THUNITS|CONDOS| APT|TOTUNITS|OFFICE|RETAIL|HOTELRMS|PARKING|OPENSP|ACRESITE|    

In [20]:
npa_data_filteredbyyear_df = npa_data_df2.filter("year >= '2014'")
drop_list = []
drop_list.append("_c0")
drop_list.append("Unnamed: 0")
npa_data_filteredbyyear_df = npa_data_filteredbyyear_df.drop(*drop_list)

In [21]:
print(npa_data_filteredbyyear_df.count())
npa_data_filteredbyyear_df.show()

1757
+------------------+---+--------------------+--------+--------+--------------------+--------+---------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------+-----------------+-------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+---------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----+----------------+
|         npa_acres|NPA|        npa_geometry|OBJECTID|Petition|          Petitioner|FromZone|   ToZone|Type|SPA|Overlay|INNOV|              Acres|           PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ|     CCW|ServiceArea|        LuseStaff|  UrbanDesign|       RezoneStaff|Consistent|ProtestPet|            Received|            Approved|Decision|           Hyperlink|Corrective| Prop

In [23]:
#Import Character.csv file
url_3 = "https://charlotte-rezoning-model.s3.us-east-2.amazonaws.com/Transportation.csv"
spark.sparkContext.addFile(url_3)
character_data_df = spark.read.csv(SparkFiles.get("Transportation.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
character_data_df.show()

+---+---+----+-----------------------+-----------------+-----------------------+---------------------+--------------+-------------------+------------+----------------+--------------------+-----------------+
|_c0|NPA|year|transit_proximate_units|transit_ridership|transit_ridership_total|sidewalk_availability|sidewalk_miles|street_connectivity|long_commute|long_commute_moe|bicycle_friendliness|transit_proximity|
+---+---+----+-----------------------+-----------------+-----------------------+---------------------+--------------+-------------------+------------+----------------+--------------------+-----------------+
|  0|  2|2017|                 1079.0|             null|                   null|                 null|          null|                1.2|        39.0|            12.0|                 1.5|            100.0|
|  1|  2|2018|                 1083.0|             null|                   null|                 null|          null|                1.2|        null|            null|     

In [24]:
joined_df = npa_data_filteredbyyear_df.join(character_data_df,on=["NPA","year"], how="left")

In [ ]:
joined_df.show()

+---+----+------------------+--------------------+--------+--------+--------------------+--------+---------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------+-----------------+-------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+---------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----------------+----+-------------------------+-------------------+-----------------------+--------------------+--------------------+------------------------------+--------------------------+-----------------------+--------------------+-----------------------------+-------------------------+-------------------+----------------+--------------------------+-----------------------+-------------------+
|NPA|y

In [25]:
def dropDupeDfCols(df):
    newcols = []
    dupcols = []

    for i in range(len(df.columns)):
        if df.columns[i] not in newcols:
            newcols.append(df.columns[i])
        else:
            dupcols.append(i)

    df = df.toDF(*[str(i) for i in range(len(df.columns))])
    for dupcol in dupcols:
        df = df.drop(str(dupcol))

    return df.toDF(*newcols)

In [26]:
clean_df = dropDupeDfCols(joined_df)
print(clean_df.count())
print(joined_df.count())

1757
1757


In [29]:
column_null_count_df = clean_df.select([count(when(isnull(c), c)).alias(c) for c in clean_df.columns]).show()

+---+----+---------+------------+--------+--------+----------+--------+------+----+---+-------+-----+-----+--------+-----+-----+---------+---------+----+---+-----------+---------+-----------+-----------+----------+----------+--------+--------+--------+---------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+-----------+-------------+--------+-----------+----------------+---+-----------------------+-----------------+-----------------------+---------------------+--------------+-------------------+------------+----------------+--------------------+-----------------+
|NPA|year|npa_acres|npa_geometry|OBJECTID|Petition|Petitioner|FromZone|ToZone|Type|SPA|Overlay|INNOV|Acres|PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ|CCW|ServiceArea|LuseStaff|UrbanDesign|RezoneStaff|Consistent|ProtestPet|Received|Approved|Decision|Hyperlink|Corrective|Propluse|AdminAppr|AdminApprD|AdminApprP|AdminApprC|SFLOTS

In [30]:
# dropping columsn with null counts above 1500
clean_columns1_df = clean_df.select("NPA","year","npa_acres","npa_geometry","OBJECTID","Petition","Petitioner","FromZone","ToZone","Type","SPA","Overlay","INNOV","Acres","PetAcres","Water","Sewer","CityLimit","ZoningMap","CCW","ServiceArea","LuseStaff","UrbanDesign","RezoneStaff","ProtestPet","Received","Approved","Decision","Hyperlink","Propluse","SHAPESTArea","SHAPESTLength","centroid","NPA_Located","nullDecisionFlag","_c0","transit_proximate_units","transit_ridership","transit_ridership_total","street_connectivity","long_commute","long_commute_moe","bicycle_friendliness","transit_proximity")

clean_columns1_df.filter("Decision <> 'App'").show(200)

+---+----+------------------+--------------------+--------+--------+--------------------+----------+------------+----+---+-------------+-----+------------------+------------------+-----+-----+---------+----------+--------------------+-----------+-----------------+-----------------+------------------+----------+--------------------+--------------------+--------+--------------------+--------+------------------+------------------+--------------------+-----------+----------------+----+-----------------------+-----------------+-----------------------+-------------------+------------+----------------+--------------------+-----------------+
|NPA|year|         npa_acres|        npa_geometry|OBJECTID|Petition|          Petitioner|  FromZone|      ToZone|Type|SPA|      Overlay|INNOV|             Acres|          PetAcres|Water|Sewer|CityLimit| ZoningMap|                 CCW|ServiceArea|        LuseStaff|      UrbanDesign|       RezoneStaff|ProtestPet|            Received|            Approved|Deci